In [2]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import bs4


In [3]:
# 
year = '2002'
date = '20020501'

response = requests.get('https://www.nrc.gov/reading-rm/doc-collections/event-status/reactor-status/' + year + '/' + date + 'ps.html')

print(response.text)

<!DOCTYPE html>
<html   lang="en-US" >
<head id="head"><title>
	Power Reactor Status Report for May 1, 2002 | NRC.gov
</title><meta charset="UTF-8" /> 
<script async type="text/javascript" id="_fed_an_ua_tag" src="https://dap.digitalgov.gov/Universal-Federated-Analytics-Min.js?agency=NRC"></script>

<meta name="apple-mobile-web-app-title" content="NRC Web">
<meta name="application-name" content="NRC Web">
<meta name="msapplication-TileColor" content="#ffffff">
<meta name="theme-color" content="#ffffff">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<meta property="og:image:width" content="279">
<meta property="og:image:height" content="279">
<meta property="og:title" content="Power Reactor Status Report for May 1, 2002">
 
<meta property="og:image" content='/cdn/site/logo/og-image.jpg'>

<meta property="og:description" content="Power Reactor Status Report for May 1, 2002">
<meta property="og:url" content="ht

In [4]:
pip show lxml


Name: lxml
Version: 6.0.0
Summary: Powerful and Pythonic XML processing library combining libxml2/libxslt with the ElementTree API.
Home-page: https://lxml.de/
Author: lxml dev team
Author-email: lxml@lxml.de
License: BSD-3-Clause
Location: /home/ballaprr/Documents/NuclearTimeSeries/env/lib/python3.12/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup
import lxml
def fetch_nrc_reactor_status(year, date):
    url = f"https://www.nrc.gov/reading-rm/doc-collections/event-status/reactor-status/{year}/{date}ps.html"
    try:
        response = requests.get(url)
        response.raise_for_status()
    except Exception as e:
        print(f"Failed to fetch data: {e}")
        return None

    soup = BeautifulSoup(response.text, 'lxml')

    all_rows = []
    for table in soup.find_all('table', class_='power'):
        rows = table.find_all('tr')[1:]  # Skip header
        for row in rows:
            cells = [td.get_text(strip=True) for td in row.find_all('td')]
            if len(cells) == 6:
                all_rows.append(cells)

    df = pd.DataFrame(all_rows, columns=['Unit', 'Power', 'Down', 'Reason', 'Change', 'Scrams'])

    df.replace('', pd.NA, inplace=True)


    return df

# Example usage
df = fetch_nrc_reactor_status('2025', '20250702')
if df is not None:
    print(df)


                    Unit Power  Down Reason Change Scrams
0        Beaver Valley 1   100  <NA>   <NA>   <NA>   <NA>
1        Beaver Valley 2    99  <NA>   <NA>   <NA>   <NA>
2       Calvert Cliffs 1   100  <NA>   <NA>   <NA>   <NA>
3       Calvert Cliffs 2   100  <NA>   <NA>   <NA>   <NA>
4            FitzPatrick   100  <NA>   <NA>   <NA>   <NA>
..                   ...   ...   ...    ...    ...    ...
89  River Bend Station 1   100  <NA>   <NA>   <NA>   <NA>
90         South Texas 1   100  <NA>   <NA>   <NA>   <NA>
91         South Texas 2   100  <NA>   <NA>   <NA>   <NA>
92           Waterford 3    91  <NA>   <NA>   <NA>   <NA>
93          Wolf Creek 1   100  <NA>   <NA>   <NA>   <NA>

[94 rows x 6 columns]
